In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from efficientnet_pytorch import EfficientNet
from PIL import Image
import numpy as np
import pandas as pd
import os
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [3]:
# Transformations
transform = A.Compose([
    A.Resize(256, 256),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
], is_check_shapes=False)

In [4]:
# Custom Dataset
class FloodDataset(Dataset):
    def __init__(self, image_dir, mask_dir, csv_file, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.metadata = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.metadata.iloc[idx, 0])
        mask_name = os.path.join(self.mask_dir, self.metadata.iloc[idx, 1])
        
        image = np.array(Image.open(img_name).convert("RGB"))
        mask = np.array(Image.open(mask_name).convert("L"))
        
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
        
        mask = mask / 255.0  # Normalize mask to [0, 1]
        return image, mask

In [10]:
# Data Loader
image_dir = 'Image'
mask_dir = 'Mask'
csv_file = 'metadata.csv'

dataset = FloodDataset(image_dir, mask_dir, csv_file, transform)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [11]:
# Model
class SegmentationModel(nn.Module):
    def __init__(self, backbone):
        super(SegmentationModel, self).__init__()
        self.backbone = backbone
        self.classifier = nn.Conv2d(backbone._fc.in_features, 1, kernel_size=1)
        self.upsample = nn.Upsample(scale_factor=32, mode='bilinear', align_corners=True)

    def forward(self, x):
        features = self.backbone.extract_features(x)
        out = self.classifier(features)
        out = self.upsample(out)
        return out

In [12]:
# Using a pretrained EfficientNet-B7 model
backbone = EfficientNet.from_pretrained('efficientnet-b3')
model = SegmentationModel(backbone)

# Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Reduced learning rate

num_epochs = 20
checkpoint_dir = './checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

Loaded pretrained weights for efficientnet-b3


In [13]:
def calculate_accuracy(outputs, masks):
    outputs = torch.sigmoid(outputs)  # Apply sigmoid to get probabilities
    preds = outputs > 0.5  # Convert probabilities to binary predictions
    correct = (preds == masks).float()  # Compare predictions with masks
    accuracy = correct.sum() / correct.numel()  # Calculate accuracy
    return accuracy.item()

In [14]:
def save_checkpoint(epoch, model, optimizer, loss, checkpoint_dir):
    checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch}.pth')
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, checkpoint_path)

In [16]:
for epoch in range(1, num_epochs + 1):
    model.train()
    running_loss = 0.0
    running_accuracy = 0.0
    
    for images, masks in dataloader:
        images = images.to(device)
        masks = masks.to(device).unsqueeze(1).float()

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        running_accuracy += calculate_accuracy(outputs, masks)
    
    epoch_loss = running_loss / len(dataloader)
    epoch_accuracy = running_accuracy / len(dataloader)
    print(f"Epoch {epoch}/{num_epochs}, Loss: {epoch_loss}, Accuracy: {epoch_accuracy}")
    
    # Save checkpoint every 5 epochs
    if epoch % 5 == 0:
        save_checkpoint(epoch, model, optimizer, epoch_loss, checkpoint_dir)
        # Ask user if they want to continue
        continue_training = input(f"Training reached epoch {epoch}. Do you want to continue? (yes/no): ")
        if continue_training.lower() != 'yes':
            print("Training stopped by user.")
            break

print("Training Complete!")

Epoch 1/20, Loss: 0.37846991097604904, Accuracy: 0.8196677646121463
Epoch 2/20, Loss: 0.3713189733994974, Accuracy: 0.8244869644577438
Epoch 3/20, Loss: 0.3581811165487444, Accuracy: 0.831235782520191
Epoch 4/20, Loss: 0.34837783268980077, Accuracy: 0.835702844568201
Epoch 5/20, Loss: 0.3470550018387872, Accuracy: 0.8347592224945893
Epoch 6/20, Loss: 0.33909591146417567, Accuracy: 0.839314434979413
Epoch 7/20, Loss: 0.33336026120830226, Accuracy: 0.8420373040276605
Epoch 8/20, Loss: 0.32428288620871465, Accuracy: 0.8473150923445418
Epoch 9/20, Loss: 0.3239408739515253, Accuracy: 0.8468552666741449
Epoch 10/20, Loss: 0.3166873261735246, Accuracy: 0.8489612888645481
Epoch 11/20, Loss: 0.3112920417978957, Accuracy: 0.8515611081509977
Epoch 12/20, Loss: 0.3161034950533429, Accuracy: 0.8503168982428473
Epoch 13/20, Loss: 0.3103532033997613, Accuracy: 0.8525498364422772
Epoch 14/20, Loss: 0.3206770375773713, Accuracy: 0.8489866514463682
Epoch 15/20, Loss: 0.30624027751587535, Accuracy: 0.854